In [1]:
! git clone https://github.com/OldBonhart/EDA-NLP-PubMed-and-bioRxiv.git
%cd EDA-NLP-PubMed-and-bioRxiv/
%cd source

Cloning into 'EDA-NLP-PubMed-and-bioRxiv'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 142 (delta 3), reused 3 (delta 0), pack-reused 132
Receiving objects: 100% (142/142), 331.78 MiB | 35.54 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Checking out files: 100% (111/111), done.
/content/EDA-NLP-PubMed-and-bioRxiv
/content/EDA-NLP-PubMed-and-bioRxiv/source


In [4]:
from google.colab import files
files.upload()

Saving mask.png to mask.png


{'mask.png': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04q\x00\x00\x03Q\x08\x06\x00\x00\x00d\xe3 \xf9\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x00\x04gAMA\x00\x00\xb1\x8f\x0b\xfca\x05\x00\x00\x00\tpHYs\x00\x00\x0e\xc3\x00\x00\x0e\xc3\x01\xc7o\xa8d\x00\x00]\x18IDATx^\xed\xf8\x8bn,\xab\x9a\xb5\xdd~\xf7\x7f\xd3\xff\xae\x85\x99\xdb\xc3\xb8\xdb\xceC\x1c\xe0\xa55\xe9\x91P\xd5\x98\xe9\x08\x02\x82\xcc\xf5\xff\xfe?N\xf7\xff\xfe\xdf\xff\x93$I\x92$Iz\xaf\xfe\xbf3\x00\x00\x00\x000)\xff#\x0e\x00\x00\x00\xc0\x02\xfc\x8f8\x00\x00\x00\x00\x0b\xf0?\xe2\x00\x00\x00\x00,\xc0\xff\x88\x03\x00\x00\x00\xb0\x00\xff#\x0e\x00\x00ok_*O\x08\x00\xf8\xd4\xce\xc7>\x06\x00\x80\x87\x8c\xffc\xcbU\x01\xc0\xce\xday\xd8\xc7\x00\x00\xf0\xab\xf1\x7fT\xb9;\x00\xd8I;\xff\xfa\x18\x00\x960\xfe\x88{7 K\xfbe\xb6\x00`\'\xed\xfc\xebc\x00\x98\xc6\xbf?\xce\xfe\xfd\xc1vE\xb0\xab\xb4\x1fV\t\x00v\xd0\xce\xbd>\x06\x80\xdb\x8d?\xcc\xee\x0cv\x91\xd6\xff\xaa\x01@e\xed\xbc\xebc\x00\xb8\xcd\xf8Cl\xa6\xa0\xaa\xb4\xde+\x04\x00U\xb5\

In [0]:
# Pubmed url
GETURL1 = "https://www.ncbi.nlm.nih.gov/pubmed/?term=Coronavirus+2019-nCoV"
NUM_PAGES1 = 1

# Biorxiv url
GETURL2 = "https://www.biorxiv.org/search/Coronavirus%252B2019-nCoV%20numresults%3A75%20sort%3Arelevance-rank?page="
NUM_PAGES2 = 1

MASK = "mask.png"
PROMO = "promo.jpg"
THEME = "Coronavirus 2019-nCoV"

In [6]:
!pip install fpdf aiohttp -qq

     |████████████████████████████████| 1.2MB 11.4MB/s 
     |████████████████████████████████| 256kB 26.6MB/s 
     |████████████████████████████████| 153kB 32.4MB/s 


In [7]:
%%time

import asyncio
import aiohttp
import socket

#import requests
from bs4 import BeautifulSoup


#GETURL = "https://www.ncbi.nlm.nih.gov/pubmed/?term=coronavirus+2019-nCoV"
POSTURL = "https://www.ncbi.nlm.nih.gov/pubmed/"


PMID = []
#NUM_PAGES = 1#46
ITEMS_PER_PAGE = 100


pubmed_MAP = {}
SEMA = asyncio.BoundedSemaphore(300)

####################### Get all PMID #########################

async def get_session_and_inputs(page, items_per_page):

    headers = {"User-Agent" : "Mozilla/5.0", "Connection": "close"}
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.get(GETURL1) as response:
            data = await response.text()
            soup = BeautifulSoup(data, "lxml")

            inputs = {i['name']: i.get('value', '') for i
                      in soup.select('form#EntrezForm input[name]')}
            inputs['EntrezSystem2.PEntrez.PubMed.Pubmed_ResultsPanel.Pubmed_DisplayBar.PageSize'] = ITEMS_PER_PAGE
            inputs['EntrezSystem2.PEntrez.PubMed.Pubmed_ResultsPanel.Pubmed_DisplayBar.PrevPageSize'] = ITEMS_PER_PAGE
            inputs['EntrezSystem2.PEntrez.DbConnector.Cmd'] = 'PageChanged'

            inputs['EntrezSystem2.PEntrez.PubMed.Pubmed_ResultsPanel.Pubmed_Pager.CurrPage'] = page
            inputs['EntrezSystem2.PEntrez.PubMed.Pubmed_ResultsPanel.Pubmed_Pager.cPage'] = page

        async with session.post(POSTURL, data=inputs) as response2:
            data = await response2.text()
            soup = BeautifulSoup(data, "lxml")
            PMID.append([pmid.text for pmid in soup('dd')])


async def get_all_pmid():
    tasks = []
    for page in range(NUM_PAGES1):
        task = asyncio.ensure_future(get_session_and_inputs(page=page,
                                                            items_per_page=NUM_PAGES1))
        tasks.append(task)

    await asyncio.gather(*tasks)

############# Get all data in MAP from each uniq PMID articles ###################

async def parse_text_from_paper(pmid):
    '''
    From link to science paper (pmid) - get html,
    then get document's body (data) from one page.
    '''
    
    conn = aiohttp.TCPConnector(family=socket.AF_INET)
    headers={"User-Agent" : "Mozilla/5.0",
             "Connection": "close"}
    
    async with aiohttp.ClientSession(headers=headers, connector=conn) as session:
        async with SEMA, session.get(f"https://www.ncbi.nlm.nih.gov/pubmed/{pmid}") as response:
            data = await response.text()
            soup = BeautifulSoup(data, "lxml")

            title = [text.find('h1').text for text in soup.find_all("div", class_="rprt_all")]
            data = [text.find('p').text for text in soup.find_all("div", class_="abstr")]
            pubmed_MAP[POSTURL+pmid] = data, title[0]

async def get_all_data():

    tasks = []
    pmid_items = [pmid_i for lst in PMID for pmid_i in lst]

    for pmid in pmid_items:

        task = asyncio.ensure_future(parse_text_from_paper(pmid))
        tasks.append(task)

    await asyncio.gather(*tasks)


loop = asyncio.get_event_loop()
loop.run_until_complete(get_all_pmid())
loop = asyncio.get_event_loop()
loop.run_until_complete(get_all_data())
    
print(f"Done!\nMap length: {len(pubmed_MAP)}")

Done!
Map length: 79
CPU times: user 3.37 s, sys: 81.2 ms, total: 3.45 s
Wall time: 6.15 s


In [8]:
%%time

import asyncio
import aiohttp

import requests
from bs4 import BeautifulSoup

#GETURL2 = "https://www.biorxiv.org/search/coronavirus%252B2019-nCoV?page="
MAIN_URL = "https://www.biorxiv.org"
PAPER_LINKS = []
#NUM_PAGES2 = 12
biorxiv_MAP = {}

async def fetch_links_from_page(page, session, geturl=GETURL2):
    """
    corutine parse links on paper from page
    and added there into list.
    """

    async with session.get(f"{geturl}{page}") as response:
        data = await response.text()
        soup = BeautifulSoup(data, "lxml")

        PAPER_LINKS.append([MAIN_URL + i.get("href")for i in 
                         soup.find_all(name='a',class_='highwire-cite-linked-title')])



async def get_paper_links(url):
    tasks = []
    async with aiohttp.ClientSession() as session:
        for page in range(NUM_PAGES2):
            task = asyncio.ensure_future(fetch_links_from_page(page, session, geturl=url))
            tasks.append(task)
        
        await asyncio.gather(*tasks)


async def fetch_content(url, session):
    async with session.get(url) as response:
        data = await response.text()
        soup = BeautifulSoup(data, "lxml")
        try:
            title = soup.find("h1", {"id": "page-title"}).text
            biorxiv_MAP[url] = [p.text for p in 
                            soup.find("div", {"id": "abstract-1"}).find_all('p')], title
        except:
            print(f"\n Problem with this link {url}")           
            biorxiv_MAP[url] = None




async def get_all_data():
    tasks = []

    async with aiohttp.ClientSession() as session:
        URLS = [link for lst in PAPER_LINKS for link in lst]
        for url in URLS:
            task = asyncio.ensure_future(fetch_content(url, session))
            tasks.append(task)
        await asyncio.gather(*tasks)




loop = asyncio.get_event_loop()
loop.run_until_complete(get_paper_links(GETURL2))

loop.run_until_complete(get_all_data())

CPU times: user 1.47 s, sys: 12.6 ms, total: 1.48 s
Wall time: 6.63 s


In [9]:
def remove_none(data):
    print(len(data))
    filtered = {k: v for k, v in data.items() if v is not None}
    data.clear()
    data.update(filtered)
    print(len(data))
    return data

pubmed_MAP = remove_none(pubmed_MAP)
biorxiv_MAP = remove_none(biorxiv_MAP)

79
79
42
42


In [0]:
from PIL import Image
import numpy as np

def get_mask(img_path):
    img = Image.open(img_path).convert("L")
    return np.array(img)

MASK = get_mask(MASK)

In [11]:
pubmed_MAP

{'https://www.ncbi.nlm.nih.gov/pubmed/31944312': (["Respiratory tract viral infection caused by viruses or bacteria is one of the most common diseases in human worldwide, while those caused by emerging viruses, such as the novel coronavirus, 2019-nCoV that caused the pneumonia outbreak in Wuhan, China most recently, have posed great threats to global public health. Identification of the causative viral pathogens of respiratory tract viral infections is important to select an appropriate treatment, save people's lives, stop the epidemics, and avoid unnecessary use of antibiotics. Conventional diagnostic tests, such as the assays for rapid detection of antiviral antibodies or viral antigens, are widely used in many clinical laboratories. With the development of modern technologies, new diagnostic strategies, including multiplex nucleic acid amplification and microarray-based assays, are emerging. This review summarizes currently available and novel emerging diagnostic methods for the det

In [12]:
from main import main
import nltk
import re
nltk.download('stopwords') 
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
%%time
main(p_data=pubmed_MAP,
     b_data=biorxiv_MAP, 
     p_gif_name="3d_pubmded.gif",
     b_gif_name="3d_bioxriv.gif",
     p_wc_name="pubmed_wordcloud",
     b_wc_name_mask="bioxriv_wordcloud_mask",
     b_wc_name="bioxriv_wordcloud",
     p_wc_name_mask="pubmed_wordcloud_mask",
     p_pdf_name="pubmed",
     b_pdf_name="bioxriv",
     p_site="PubMed",
     b_site="bioRxiv", 
     b_2d_name1="biorxiv_tsne_2d_dots.png",
     b_2d_name2="biorxiv_tsne_2d_words.png",
     p_2d_name1="pubmed_tsne_2d_dots.png",
     p_2d_name2="pubmed_tsne_2d_words.png", 
     b_sim_name="bioxriv_similar_words.png", 
     p_sim_name="pubmed_similar_words.png",
     theme=THEME,
     mask=MASK, 
     img_promo=PROMO)

Num articles: 79
Num senteces: 245
Num articles: 42
Num senteces: 259


/usr/local/lib/python3.6/dist-packages/matplotlib/collections.py:857: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Amount of empty data: 22
Amount of empty data: 0
Done!
CPU times: user 7min 32s, sys: 5.84 s, total: 7min 38s
Wall time: 6min 7s


In [0]:
from IPython.display import FileLink, FileLinks
FileLinks('/content/')